In [ ]:
# you may need to install pyamg
!pip install pyamg

In [ ]:
import numpy as np
import scipy.io as sio
import pyamg
import scipy.sparse.linalg as sla

import matplotlib.pyplot as plt
from matplotlib import collections
from matplotlib import tri
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# recirculating flow

In [ ]:
A = pyamg.gallery.load_example('recirc_flow')['A'].tocsr()

In [ ]:
ml = pyamg.smoothed_aggregation_solver(A)

In [ ]:
res = []
x = ml.solve(np.random.rand(A.shape[0]), residuals=res)
res = np.array(res)
res

The error explodes.  Let's see what relaxation does (or does not) do

In [ ]:
x = np.random.rand(A.shape[0])
pyamg.relaxation.relaxation.jacobi(A, x, 0*x, iterations=5, omega=4/3)
plt.pcolormesh(x.reshape(15,15))
plt.colorbar()

Since Jacobi requires (sufficient) a weakly diagonally dominant matrix, this may not be the right smoother.  Let's try Gauss-Seidel on $AA^T$

In [ ]:
x = np.random.rand(A.shape[0])
pyamg.relaxation.relaxation.gauss_seidel_ne(A, x, 0*x, iterations=5, omega=4/3)
plt.pcolormesh(x.reshape(15,15))
plt.colorbar()

In [ ]:
ml = pyamg.smoothed_aggregation_solver(A,
        presmoother=('gauss_seidel_nr', {'sweep': 'symmetric', 'iterations': 2}),
        postsmoother=('gauss_seidel_nr', {'sweep': 'symmetric', 'iterations': 2}),)

In [ ]:
res = []
x = ml.solve(np.random.rand(A.shape[0]), residuals=res)
res = np.array(res)
res[1:]/res[:-1]

At least the problem doesn't blow up.

What does the solution look like?

In [ ]:
plt.pcolormesh(x.reshape(15,15))
plt.colorbar()

# Adding better strength

In [ ]:
strength=('evolution', {'k': 2, 'proj_type': 'l2', 'epsilon': 4.0})
ml = pyamg.smoothed_aggregation_solver(A,
        presmoother=('gauss_seidel_nr', {'sweep': 'symmetric', 'iterations': 2}),
        postsmoother=('gauss_seidel_nr', {'sweep': 'symmetric', 'iterations': 2}),
                                      strength=strength)

In [ ]:
res = []
x = ml.solve(np.random.rand(A.shape[0]), residuals=res)
res = np.array(res)
res[1:]/res[:-1]

In [ ]:
plt.pcolormesh(x.reshape(15,15))

# Improving the candidate vectors

In [ ]:
improve_candidates=[('gauss_seidel_nr', {'sweep': 'symmetric', 'iterations': 4}), None, None, None, None, None, None, None, None, None, None, None, None, None, None]
ml = pyamg.smoothed_aggregation_solver(A,
        presmoother=('gauss_seidel_nr', {'sweep': 'symmetric', 'iterations': 2}),
        postsmoother=('gauss_seidel_nr', {'sweep': 'symmetric', 'iterations': 2}),
                                      strength=strength,
                                      improve_candidates=improve_candidates)

In [ ]:
res = []
x = ml.solve(np.random.rand(A.shape[0]), residuals=res)
res = np.array(res)
res[1:]/res[:-1]

In [ ]:
plt.pcolormesh(x.reshape(15,15))

# Adding energy minimization

In [ ]:
smooth=('energy', {'krylov': 'gmres', 'maxiter': 2, 'degree': 1, 'weighting': 'local'})
ml = pyamg.smoothed_aggregation_solver(A,
        presmoother=('gauss_seidel_nr', {'sweep': 'symmetric', 'iterations': 2}),
        postsmoother=('gauss_seidel_nr', {'sweep': 'symmetric', 'iterations': 2}),
                                      strength=strength,
                                      improve_candidates=improve_candidates,
                                      smooth=smooth)

In [ ]:
ml

In [ ]:
res = []
x = ml.solve(np.random.rand(A.shape[0]), residuals=res)
res = np.array(res)
res[1:]/res[:-1]

In [ ]:
plt.pcolormesh(x.reshape(15,15))

# Oh, right!  A Krylov accelerator

In [ ]:
res = []
x = ml.solve(np.random.rand(A.shape[0]), residuals=res, accel='gmres')
res = np.array(res)
res[1:]/res[:-1]

In [ ]:
res